In [33]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Check if the container is running
container_status = !docker ps -f name=browserless
is_running = any('browserless' in line for line in container_status)

if is_running:
    print("The container 'browserless' is running.")
else:
    print("The container 'browserless' is not running.")
    # Run the container if it's not running
    !docker run -d --rm --name browserless -p 3000:3000 -e "CONCURRENT=10" -e "TOKEN=${BROWSERLESS_API_KEY}" ghcr.io/browserless/chromium

The container 'browserless' is running.


In [34]:
import json
import os
import requests
from crewai import Agent, Task, Crew, LLM
from crewai_tools import tool
from unstructured.partition.html import partition_html
from textwrap import dedent

llm = LLM(
    api_key = os.getenv("AZURE_OPENAI_KEY"),
    api_version = os.getenv("AZURE_OPENAI_VERSION"),
    base_url = os.getenv("AZURE_OPENAI_ENDPOINT"),
    model = 'gpt-4o-mini',
    azure=True
)

In [39]:
@tool("Scrape website content")
def scrape_and_summarize_website(website: str) -> str:
    """Useful to scrape and summarize a website content"""
    url = f"{os.environ['BROWSERLESS_API_HOST']}/content?token={os.environ['BROWSERLESS_API_KEY']}"
    payload = json.dumps({"url": website})
    headers = {'cache-control': 'no-cache', 'content-type': 'application/json'}
    response = requests.request("POST", url, headers=headers, data=payload)
    elements = partition_html(text=response.text)
    content = "\n\n".join([str(el) for el in elements])
    content = [content[i:i + 8000] for i in range(0, len(content), 8000)]
    summaries = []
    for chunk in content:
        agent = Agent(
            role='Principal Researcher',
            goal=
            'Do amazing researches and summaries based on the content you are working with',
            backstory=
            "You're a Principal Researcher at a big company and you need to do a research about a given topic.",
            llm=llm,
            allow_delegation=False)
        task = Task(
            agent=agent,
            description=
            f'Analyze and summarize the content bellow, make sure to include the most relevant information in the summary, return only the summary nothing else.\n\nCONTENT\n----------\n{chunk}',
            expected_output='A summary of the content',
        )
        summary = task.execute_sync().raw
        summaries.append(summary)
    return "\n\n".join(summaries)


@tool("Search the internet")
def search_internet(query: str) -> str:
    """Useful to search the internet
    about a a given topic and return relevant results"""
    top_result_to_return = 4
    url = "https://google.serper.dev/search"
    payload = json.dumps({"q": query})
    headers = {
        'X-API-KEY': os.environ['SERPER_API_KEY'],
        'content-type': 'application/json'
    }
    response = requests.request("POST", url, headers=headers, data=payload)
    # check if there is an organic key
    if 'organic' not in response.json():
        return "Sorry, I couldn't find anything about that, there could be an error with you serper api key."
    else:
        results = response.json()['organic']
        string = []
        for result in results[:top_result_to_return]:
            try:
                string.append('\n'.join([
                    f"Title: {result['title']}", f"Link: {result['link']}",
                    f"Snippet: {result['snippet']}", "\n-----------------"
                ]))
            except KeyError:
                next

        return '\n'.join(string)


@tool("Make a calculation")
def calculate(operation: str):
    """Useful to perform any mathematical calculations, 
    like sum, minus, multiplication, division, etc.
    The input to this tool should be a mathematical 
    expression, a couple examples are `200*7` or `5000/2*10`
    """
    try:
        return eval(operation)
    except SyntaxError:
        return "Error: Invalid syntax in mathematical expression"

In [36]:
class TripAgents():

  def city_selection_agent(self):
    return Agent(
        role='City Selection Expert',
        goal='Select the best city based on weather, season, and prices',
        backstory=
        'An expert in analyzing travel data to pick ideal destinations',
        tools=[
            search_internet,
            scrape_and_summarize_website,
        ],
        llm=llm,
        verbose=True)

  def local_expert(self):
    return Agent(
        role='Local Expert at this city',
        goal='Provide the BEST insights about the selected city',
        backstory="""A knowledgeable local guide with extensive information
        about the city, it's attractions and customs""",
        tools=[
            search_internet,
            scrape_and_summarize_website,
        ],
        llm=llm,
        verbose=True)

  def travel_concierge(self):
    return Agent(
        role='Amazing Travel Concierge',
        goal="""Create the most amazing travel itineraries with budget and 
        packing suggestions for the city""",
        backstory="""Specialist in travel planning and logistics with 
        decades of experience""",
        tools=[
            search_internet,
            scrape_and_summarize_website,
            calculate,
        ],
        llm=llm,
        verbose=True)

In [37]:
class TripTasks:

    def identify_task(self, agent, origin, cities, interests, range):
        return Task(
            description=dedent(f"""
                Analyze and select the best city for the trip based 
                on specific criteria such as weather patterns, seasonal
                events, and travel costs. This task involves comparing
                multiple cities, considering factors like current weather
                conditions, upcoming cultural or seasonal events, and
                overall travel expenses. 
                
                Your final answer must be a detailed
                report on the chosen city, and everything you found out
                about it, including the actual flight costs, weather 
                forecast and attractions.
                {self.__tip_section()}

                Traveling from: {origin}
                City Options: {cities}
                Trip Date: {range}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Detailed report on the chosen city including flight costs, weather forecast, and attractions"
        )

    def gather_task(self, agent, origin, interests, range):
        return Task(
            description=dedent(f"""
                As a local expert on this city you must compile an 
                in-depth guide for someone traveling there and wanting 
                to have THE BEST trip ever!
                Gather information about key attractions, local customs,
                special events, and daily activity recommendations.
                Find the best spots to go to, the kind of place only a
                local would know.
                This guide should provide a thorough overview of what 
                the city has to offer, including hidden gems, cultural
                hotspots, must-visit landmarks, weather forecasts, and
                high level costs.
                
                The final answer must be a comprehensive city guide, 
                rich in cultural insights and practical tips, 
                tailored to enhance the travel experience.
                {self.__tip_section()}

                Trip Date: {range}
                Traveling from: {origin}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Comprehensive city guide including hidden gems, cultural hotspots, and practical travel tips"
        )

    def plan_task(self, agent, origin, interests, range):
        return Task(
            description=dedent(f"""
                Expand this guide into a full 7-day travel 
                itinerary with detailed per-day plans, including 
                weather forecasts, places to eat, packing suggestions, 
                and a budget breakdown.
                
                You MUST suggest actual places to visit, actual hotels 
                to stay and actual restaurants to go to.
                
                This itinerary should cover all aspects of the trip, 
                from arrival to departure, integrating the city guide
                information with practical travel logistics.
                
                Your final answer MUST be a complete expanded travel plan,
                formatted as markdown, encompassing a daily schedule,
                anticipated weather conditions, recommended clothing and
                items to pack, and a detailed budget, ensuring THE BEST
                TRIP EVER. Be specific and give it a reason why you picked
                each place, what makes them special! {self.__tip_section()}

                Trip Date: {range}
                Traveling from: {origin}
                Traveler Interests: {interests}
            """),
            agent=agent,
            expected_output="Complete expanded travel plan with daily schedule, weather conditions, packing suggestions, and budget breakdown"
        )

    def __tip_section(self):
        return "If you do your BEST WORK, I'll tip you $100!"

In [40]:
class TripCrew:

  def __init__(self, origin, cities, date_range, interests):
    self.cities = cities
    self.origin = origin
    self.interests = interests
    self.date_range = date_range

  def run(self):
    agents = TripAgents()
    tasks = TripTasks()

    city_selector_agent = agents.city_selection_agent()
    local_expert_agent = agents.local_expert()
    travel_concierge_agent = agents.travel_concierge()

    identify_task = tasks.identify_task(
      city_selector_agent,
      self.origin,
      self.cities,
      self.interests,
      self.date_range
    )
    gather_task = tasks.gather_task(
      local_expert_agent,
      self.origin,
      self.interests,
      self.date_range
    )
    plan_task = tasks.plan_task(
      travel_concierge_agent, 
      self.origin,
      self.interests,
      self.date_range
    )

    crew = Crew(
      agents=[
        city_selector_agent, local_expert_agent, travel_concierge_agent
      ],
      tasks=[identify_task, gather_task, plan_task],
      verbose=True
    )

    result = crew.kickoff()
    return result

location = "Shanghai"
cities = "London"
date_range = "2025-01-01 to 2025-01-07"
interests = "Museums, Parks, Restaurants"

trip_crew = TripCrew(location, cities, date_range, interests)
result = trip_crew.run()

2024-11-07 19:07:10,805 - 140490531176576 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: City Selection Expert
## Task: 
Analyze and select the best city for the trip based 
on specific criteria such as weather patterns, seasonal
events, and travel costs. This task involves comparing
multiple cities, considering factors like current weather
conditions, upcoming cultural or seasonal events, and
overall travel expenses. 

Your final answer must be a detailed
report on the chosen city, and everything you found out
about it, including the actual flight costs, weather 
forecast and attractions.
If you do your BEST WORK, I'll tip you $100!

Traveling from: Shanghai
City Options: London
Trip Date: 2025-01-01 to 2025-01-07
Traveler Interests: Museums, Parks, Restaurants



# Agent: City Selection Expert
## Thought: I need to gather detailed information about London, focusing on flight costs from Shanghai, the weather forecast for the specified dates, and attractions relevant to the traveler's interests in museums, parks, and restaurants.
## Using tool: Search the internet

In [ ]:
from IPython.display import Markdown, display
display(Markdown(result.raw))

```markdown
# 7-Day London Itinerary (January 1 - January 7, 2025)

## Overview
London, rich in history and bustling with interesting activities, offers the perfect blend of museums, parks, and exceptional dining. Below is a complete itinerary designed for an unforgettable experience.

## Packing Suggestions
- Warm clothing: sweaters, thermal layers, gloves, and hats.
- Waterproof jacket and an umbrella.
- Comfortable walking shoes.
- Portable charger and camera for capturing memories.

## Weather Forecast
- Daily temperatures will range from **3°C to 7°C (37°F to 44°F)** with a high chance of rain (about 50% of days).
- Expect cloudy skies and dress in layers for warmth.

### Budget Breakdown
| Item                       | Cost (per person) |
|----------------------------|------------------|
| Flights (Round-trip)       | $585 - $700      |
| Hotel (7 nights)           | $850 - $2100     |
| Meals (approx. $40/day)   | $280             |
| Transportation (Oyster Card)| $40              |
| Attractions (avg. per day) | $100             |
| **Total Estimated Cost**    | **$1895 - $3230**|

---

## Daily Itinerary

### Day 1: Arrival & New Year’s Day Parade
- **Morning:** Arrive at London Heathrow Airport.
- **Hotel Check-In:** *The Resident Covent Garden*.
- **Afternoon:** Head to the New Year's Day Parade showcasing colorful floats and performances along Piccadilly.
- **Dinner:** *Dishoom* for authentic Indian cuisine.
  
### Day 2: The British Museum & Bloomsbury
- **Morning:** Visit *The British Museum* (free entry).
- **Lunch:** Enjoy a meal at *The Great Court Restaurant* inside the museum.
- **Afternoon:** Stroll through Bloomsbury; visit *Russell Square* and the nearby *Bloomsbury Bookshop*.
- **Dinner:** Try *Sketch* for its quirky decor and signature dishes.

### Day 3: Natural History Museum & Kensington Gardens 
- **Morning:** Visit *The Natural History Museum* (free entry).
- **Lunch:** Approach *The V&A Café* in the nearby Victoria and Albert Museum.
- **Afternoon:** Walk through *Kensington Gardens*, visit the *Serpentine Gallery*.
- **Dinner:** *Hawksmoor* for high-quality British steak.

### Day 4: Camden Market & Regent's Park
- **Morning:** Explore *Camden Market* for food stalls and unique shops.
- **Lunch:** Choose various street foods around the market.
- **Afternoon:** Enjoy a leisurely walk in *Regent’s Park*, visiting the rose gardens.
- **Dinner:** Casual dining at *Pizza Union* nearby.

### Day 5: Columbia Road Flower Market & Afternoon Tea
- **Morning:** Visit *Columbia Road Flower Market* (Sunday only).
- **Lunch:** Quick café stop at *Laxe’s Café* for light bites.
- **Afternoon:** Book an afternoon tea at *Petersham Nurseries* in Richmond.
- **Dinner:** Explore local pubs for traditional English fare.

### Day 6: Shoreditch & Chelsea 
- **Morning:** Venture into *Shoreditch* for street art and boutique shops.
- **Lunch:** Stop by *Dishoom Shoreditch* for lunch.
- **Afternoon:** Head to *Chelsea* for shopping and explore the quiet streets.
- **Dinner:** Book a table at the *Bluebird Chelsea*. 

### Day 7: Departure
- **Morning:** Last-minute shopping or visits near your hotel.
- **Lunch:** Simple, hearty breakfast at hotel before departing.
- **Afternoon:** Depart for Heathrow Airport.

---

This itinerary introduces a mix of cultural heritage, relaxation in beautiful parks, culinary delights, and local experiences, ensuring your trip to London is memorable and enriching!
```